In [ ]:
def range_doppler_process(frame,range_zero_pad,doppler_zero_pad):
    if not isinstance(frame,fr.Frame):
        print("Argument of algo_process() can only be a Frame instance")
        return
    #Fast time Windowing
    tmp_frame = frame.chirp
    window = np.hamming(frame.sam_x_chir)
    tmp_frame = apply_window(tmp_frame, window,'fast')
    #Zero-pad & Fast FFT
    fast_fft_data_out = fftn(np.pad(tmp_frame,((0,range_zero_pad-frame.sam_x_chir),(0,0),(0,0))), axes=0)/np.sqrt(range_zero_pad)

    for antenna in range(frame.num_antennas):
        for slow_time_row in range(range_zero_pad):
            fast_fft_data_out[slow_time_row,antenna,:] -= np.mean(fast_fft_data_out[slow_time_row,antenna,:])
    
    #Slow time Windowing
    tmp_frame = apply_window(fast_fft_data_out - np.mean(fast_fft_data_out, axis=0), np.blackman(frame.chir_x_fr),'slow')
    #Zero-pad & Slow FFT
    slow_fft_data_out = fftn(np.pad(tmp_frame,((0,0),(0,0),(0,doppler_zero_pad-frame.chir_x_fr))), axes=2)/np.sqrt(doppler_zero_pad)
    #Return range-doppler map
    return slow_fft_data_out

In [ ]:
def apply_window(data_in, window, direction):
    if direction == 'fast':
        return (data_in.T*window).T
    if direction == 'slow':
        return (data_in*window)